# Retrieval-Augmented Generation (RAG) Notebook

Welcome to this **RAG** (Retrieval-Augmented Generation) tutorial! 

The goal of RAG is to combine the power of:
1. **Language Models** (for natural language understanding and generation), and 
2. **Vector Databases** (for relevant context retrieval)

We'll embed our data, store it in an in-memory vector DB, retrieve the most relevant text chunks based on a query, 
and finally pass those chunks as context to a language model to get a higher-quality, context-aware answer.

---
**Notebook Overview**:
1. **Installing Dependencies**  
2. **Loading Dataset**  
3. **Initializing Models** (embedding model + LLM for generation)  
4. **Building a Simple In-Memory Vector Database**
5. **Retrieving Relevant Data**
6. **Generating an Answer using the Retrieved Chunks**  

Let's start!

# 1. Installing Dependencies

We'll also download the example model files to use for embeddings and LLM.

In [2]:
!pip install llama-index-llms-llama-cpp transformers
!wget https://huggingface.co/CompendiumLabs/bge-base-en-v1.5-gguf/resolve/main/bge-base-en-v1.5-q4_k_m.gguf

Defaulting to user installation because normal site-packages is not writeable
  Using cached llama_index_llms_llama_cpp-0.4.0-py3-none-any.whl (7.5 kB)
  Using cached llama_index_core-0.12.15-py3-none-any.whl (1.6 MB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl (25 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
  Using cached marshmallow-3.26.0-py3-none-any.whl (50 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
--2025-02-03 12:50:50--  https://huggingface.co/CompendiumLabs/bge-base-en-v1.5-gguf/resolve/main/bge-base-en-v1.5-q4_k_m.gguf
Resolving huggingface.co (huggingface.co)... 18.165.122.101, 18.165.122.30, 18.

# 2. Loading Dataset
In this section, we'll download a simple file containing *facts about cats*. 
Then we'll read it into a list of text lines. We'll limit the data to the first 10 lines for this demo.

In [ ]:
# Let's download the file
!wget https://raw.githubusercontent.com/draios/bashbot-scripts/refs/heads/master/cat-facts.txt

In [45]:
dataset = []
with open('cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')

Loaded 319 entries


In [46]:
# For demonstration, we'll only keep the first 10 lines
dataset = dataset[:10]

In [47]:
# Let's print them out to see what we have
for i, document in enumerate(dataset):
    print(f'Doc #{i} : {document.strip()}')

Doc #0 : On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.
Doc #1 : Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.
Doc #2 : When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.
Doc #3 : The technical term for a cat’s hairball is a “bezoar.”
Doc #4 : A group of cats is called a “clowder.”
Doc #5 : Female cats tend to be right pawed, while male cats are more often left pawed. Interestingly, while 90% of humans are right handed, the remaining 10% of lefties also tend to be male.
Doc #6 : A cat can’t climb head first down a tree because every claw on a cat’s paw points the same way. To get down from a tree, a cat must back down.
Doc #7 : Cats make about 100 different sounds. Dogs make only about 10.
Doc #8 : A cat’s brain is biologically more similar to a human brain than it is to a dog’s. Both human

# 3. Initializing Models
We'll load two models from the `llama-cpp` library:
1. An **embedding model** for converting text into numerical vectors.
2. A **language model** (LLM) for generating or completing text.

---
**Important**: 
- The first call uses `Llama(..., embedding=True)` to instantiate an embedding model.
- The second call uses `Llama.from_pretrained(...)` to load a separate model for text generation.

In [19]:
import transformers
from llama_cpp import Llama

In [48]:
# Embedding model file (downloaded above)
EMBEDDING_MODEL_FILE = "bge-base-en-v1.5-q4_k_m.gguf"

# Example LLM model from Hugging Face / local file
# (You might need to replace 'repo_id' and 'filename' with valid references)
LLM_REPO_ID = "bartowski/Llama-3.2-1B-Instruct-GGUF"
LLM_FILENAME = "Llama-3.2-1B-Instruct-IQ3_M.gguf"

In [50]:
print("Loading the embedding model.")
embedding_model = Llama(
    EMBEDDING_MODEL_FILE, 
    embedding=True
)

print("Loading the language model for generation.")
llm = Llama.from_pretrained(
    repo_id=LLM_REPO_ID,
    filename=LLM_FILENAME
)

print("Models loaded successfully!")

llama_model_loader: loaded meta data with 22 key-value pairs and 197 tensors from bge-base-en-v1.5-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = bge-base-en-v1.5
llama_model_loader: - kv   2:                           bert.block_count u32              = 12
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 768
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 3072
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f32   

Loading the embedding model.
Loading the language model for generation.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.quantization_version': '2', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.seperator_token_id': '102', 'bert.context_length': '512', 'bert.feed_forward_length': '3072', 'bert.block_count': '12', 'general.architecture': 'bert', 'bert.embedding_length': '768', 'general.name': 'bge-base-en-v1.5', 'bert.attention.layer_norm_epsilon': '0.000000', 'tokenizer.ggml.model': 'bert', 'general.file_type': '15', 'bert.attention.causal': 'false', 'bert.pooling_type': '2', 'tokenizer.ggml.eos_token_id': '102', 'tokenizer.ggml.token_type_count': '2', 'tokenizer.ggml.bos_token_id': '101', 'bert.attention.head_count': '12', 'tokenizer.ggml.unknown_token_id': '100'}
Using fallback chat format: llama-2
llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from /users/attiehjo/.cache/huggingface/hub/models--bartowski--Llama-3.2-1

Models loaded successfully!


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'quantize.imatrix.entries_count': '112', 'quantize.imatrix.dataset': '/training_dir/calibration_datav3.txt', 'general.quantization_version': '2', 'tokenizer.ggml.eos_token_id': '128009', 'tokenizer.ggml.pre': 'llama-bpe', 'tokenizer.chat_template': '{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%

# Quick Test of Embeddings

Let's do a quick sanity check of the embedding model with a small piece of text. 
If it works, we'll see a list of numbers (the embedding vector). 
The exact length of the vector depends on the model.

In [51]:
test_text = ["hello world"]
test_embedding = embedding_model.embed(test_text)
print(f"Embedding vector length: {len(test_embedding[0])}")
print("Sample of the embedding vector:", test_embedding[0][:10], "...")  # print first 10 floats

llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     358.04 ms /     4 tokens (   89.51 ms per token,    11.17 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     359.46 ms /     5 tokens


Embedding vector length: 768
Sample of the embedding vector: [-0.1547335833311081, 1.0392863750457764, 0.3050381541252136, 0.24297626316547394, 0.4276876747608185, -0.05202116817235947, 0.2630404829978943, 0.5674030780792236, -0.15648683905601501, -0.7434648871421814] ...


# 4. Building a Simple In-Memory Vector Database

We'll store each text chunk along with its embedding in a Python list, `VECTOR_DB`.
The structure will be a list of `(chunk_text, embedding_vector)` tuples.

---
**Implementation Steps**:
1. Go through each text chunk in `dataset`.
2. Generate its embedding.
3. Store (chunk, embedding) in `VECTOR_DB`.

In [52]:
VECTOR_DB = []

for i, chunk in enumerate(dataset):
    # We pass a list containing a single document to `embed`
    embedding = embedding_model.embed([chunk])[0]  
    VECTOR_DB.append((chunk, embedding))
    print(f'Added chunk {i+1}/{len(dataset)} to the database.')

llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     469.33 ms /    35 tokens (   13.41 ms per token,    74.57 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     470.88 ms /    36 tokens


Added chunk 1/10 to the database.


llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     439.27 ms /    27 tokens (   16.27 ms per token,    61.47 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     440.69 ms /    28 tokens


Added chunk 2/10 to the database.


llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     429.08 ms /    25 tokens (   17.16 ms per token,    58.26 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     430.43 ms /    26 tokens


...

Added chunk 8/10 to the database.


llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     490.72 ms /    41 tokens (   11.97 ms per token,    83.55 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     492.08 ms /    42 tokens


Added chunk 9/10 to the database.


llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     411.13 ms /    20 tokens (   20.56 ms per token,    48.65 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     412.61 ms /    21 tokens


Added chunk 10/10 to the database.


# 5. **Retrieving Relevant Data**  
## 5.1 Defining a Similarity Metric

We'll use **cosine similarity** to measure how close two vectors are.  
Cosine similarity is a common metric for embeddings, defined as:  

\\[
    \text{similarity}(A, B) = 
    \frac{A \cdot B}{\|A\| \times \|B\|}
\]

We'll write a small helper function to compute this.


In [53]:
def cosine_similarity(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x ** 2 for x in a]) ** 0.5
    norm_b = sum([x ** 2 for x in b]) ** 0.5
    return dot_product / (norm_a * norm_b)

## 5.2 Creating a Retrieval Function

Given a user query, we:
1. Compute its embedding.
2. Compute its similarity to every chunk in `VECTOR_DB`.
3. Return the top N chunks (by descending similarity).

This is our basic *retrieval* step in the RAG pipeline.


In [55]:
def retrieve(query, top_n=3):
    # Embed the user query
    query_embedding = embedding_model.embed([query])[0]
    
    # Calculate similarities
    similarities = []
    for chunk_text, embedding in VECTOR_DB:
        similarity = cosine_similarity(query_embedding, embedding)
        similarities.append((chunk_text, similarity))

    # Sort by similarity in descending order
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Return the top N most relevant chunks
    return similarities[:top_n]

### Let's Test the Retrieval

We'll ask a random question about cats and see which facts get retrieved from our database.

In [56]:
# In[16]:
input_query = "Why don't cats have a sweet tooth?"
retrieved_knowledge = retrieve(input_query, top_n=3)

print("Retrieved knowledge (Top 3):")
for i, (chunk, similarity) in enumerate(retrieved_knowledge):
    print(f"\nRank {i+1} (similarity: {similarity:.2f}):")
    print(chunk.strip())

llama_perf_context_print:        load time =     359.42 ms
llama_perf_context_print: prompt eval time =     382.84 ms /    12 tokens (   31.90 ms per token,    31.34 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     384.13 ms /    13 tokens


Retrieved knowledge (Top 3):

Rank 1 (similarity: 0.87):
Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.

Rank 2 (similarity: 0.60):
A cat can’t climb head first down a tree because every claw on a cat’s paw points the same way. To get down from a tree, a cat must back down.

Rank 3 (similarity: 0.55):
Female cats tend to be right pawed, while male cats are more often left pawed. Interestingly, while 90% of humans are right handed, the remaining 10% of lefties also tend to be male.


## 5.3. Constructing an Instruction Prompt

We now combine the retrieved text into a *context* to give to our LLM.
Then we ask the user query again, instructing the model to:
- Use only the provided context
- Avoid making up new information

This prompt-based approach helps the model stay focused and reduces hallucination.

In [ ]:
# Combine the retrieved chunks into a single string
context = "\n".join(f"- {chunk.strip()}" for (chunk, _) in retrieved_knowledge)

instruction_prompt = f"""You are a helpful chatbot.  
Use only the following pieces of context to answer the question. Don't make up any new information. 

Context:
{context}
"""

print("Instruction Prompt:\n")
print(instruction_prompt)

# 6. Generating an Answer using the Retrieved Chunks

We'll feed our system prompt (with context) and user query to the LLM. 
Then, we print out the resulting answer. 

In [60]:
stream = llm.create_chat_completion(
	messages = [
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ]
)
# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream["choices"]:
  print(chunk['message']['content'])

Llama.generate: 183 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   28508.50 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    6716.71 ms /    18 runs   (  373.15 ms per token,     2.68 tokens per second)
llama_perf_context_print:       total time =    6737.74 ms /    19 tokens


Chatbot response:
Scientists believe that this is due to a mutation in a key taste receptor in cats.


# Conclusion

You've just implemented a simple Retrieval-Augmented Generation pipeline:
1. Embedding your data
2. Storing it in an in-memory vector database
3. Retrieving the most relevant chunks by cosine similarity
4. Passing these chunks as context to a language model for a final answer

This approach helps your LLM stay focused and accurate when answering questions about a specific domain. 

---